Originally generated with ChatGPT-4o on 2024-07-09, with modifications

In [73]:
from rdflib import Graph, URIRef, Literal
from rdflib.namespace import RDF
import pandas as pd
import re
import urllib.parse

In [74]:
# Create an RDF graph
g = Graph()
g.parse("../../schema/authority/general_authority_to_ric-o_model_v11_-_9_july_2024.ttl",
        format="turtle")  # Adjust the format as needed

<Graph identifier=Nb476c12bdd36492a8b6769a92b77e82e (<class 'rdflib.graph.Graph'>)>

In [75]:
# Query to get all subjects, predicates, and objects
rico = 'https://www.ica.org/standards/RiC/ontology#'
schema = 'http://gbad.archives.gov.on.ca/schema/authority#'
query = f"""
PREFIX rico: <{rico}>

SELECT ?subject ?predicate ?object
WHERE {{
  ?subject ?predicate ?object.
}}
"""
# Execute the query
result = g.query(query)

In [76]:
table = pd.read_csv('../../schema/authority/Authorities to RiC Data Elements Mapping.csv')
table.head()


,Authority Field Identifier,Authority Data Element,Authority Mnemonic,RiC-CM ID,RiC-CM Name,RiC-O Name,RiC-O IRI,RiC-O Type,Notes,Requires HTML Label?,Combine Multiple Instances?
0,AUTH01,Agency Reference Code,REFA,RiC-A22,Identifier,rico:Identifier,https://www.ica.org/standards/RiC/ontology#Ide...,Class,Use object property rico:hasIdentifierType to ...,NaN,NaN
1,AUTH02,Archivist,ARCHAU,RiC-E08,Person,rico:Person,https://www.ica.org/standards/RiC/ontology#Person,Class,Modelling the describing archivist using the c...,NaN,NaN
2,AUTH03,Authority Type,AUTHTP,RiC-A12,Corporate Body Type,rico:CorporateBodyType,https://www.ica.org/standards/RiC/ontology#Cor...,Class,RiC does not have a generic authority type ent...,NaN,NaN
3,AUTH04,Authorizing Agent,AA,RiC-E17,Mandate,rico:Mandate,https://www.ica.org/standards/RiC/ontology#Man...,Class,NaN,NaN,NaN
4,AUTH05,Biographical Sketch or Administrative History,ADM,RiC-A21,History,rico:history,https://www.ica.org/standards/RiC/ontology#his...,Datatype property,It is also possible (but more complex) to mode...,Y,Y


In [78]:
def parse_node(uri):
    parsed_result = {'element': str(uri), 'identifier': None, 'mnemonic': None, 'has_match': False}
    node = str(uri).split(schema)[-1] if str(uri).startswith(schema) else None
    if node:
        # Decode URL
        decoded_node = urllib.parse.unquote(node)
        
        # Regex to match the structure {Authority Data Element} ({Authority Field Identifier}) {Authority Mnemonic}
        match = re.match(r"(?P<element>.+)\s+\((?P<identifier>.+)\)(?P<mnemonic>.+)", decoded_node)

        if match:
            parsed_result = match.groupdict()
            parsed_result['has_match'] = not table[
                    (table['Authority Data Element'] == parsed_result['element']) &
                    (table['Authority Field Identifier'] == parsed_result['identifier']) &
                    (table['Authority Mnemonic'] == parsed_result['mnemonic'])
                ].empty
        else:
            parsed_result['element'] = decoded_node.strip()

    return parsed_result

# List to hold the parsed results
parsed_results = []

# Process the results and create new triples
for row in result:
    subject = row.subject
    predicate = row.predicate
    object = row.object

    parsed_result = parse_node(subject)
    parsed_results.append(parsed_result)
    
    # Example: create new triples by modifying the predicate or object
    #new_predicate = URIRef("http://example.org/newPredicate")
    #new_object = Literal(f"Modified: {object}")

    # Add new triples to the graph
    #g.add((subject, new_predicate, new_object))

# Save the updated graph to a file
#g.serialize(destination="updated_data.ttl", format="turtle")

In [ ]:
# Convert the parsed results to a dataframe
parsed_df = pd.DataFrame(parsed_results)

In [84]:
parsed_df[parsed_df['has_match']==False].drop_duplicates()

,element,identifier,mnemonic,has_match
1,Authority Record,None,None,False
2,http://gbad.archives.gov.on.ca/schema/authority,None,None,False
5,Date Of Subordination,AUTH08,DATECONT,False
9,Controlling Agencies,AUTH08,CONTAG,False
11,Agent Identifier,None,None,False
13,Variant Name,None,None,False
15,Parallel Name,None,None,False
16,Parallel Names,AUTH14,PAR,False
18,Dates Of Existence,AUTH10,DATEEX,False
36,Controlling Agency Relation,None,None,False
